###API KEYS

In [ ]:
# Set API KEYs

import os

os.environ["SEC_API_USER_AGENT"] =

os.environ["BEA_API_KEY"] =

os.environ["FRED_API_KEY"] =

###Install

In [ ]:
!pip install finagg
!finagg install --help

###Get Data

In [ ]:
!finagg install -ss economic -ts indices -z -r  #Download all data

In [ ]:
from sqlalchemy import create_engine
import sqlite3
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


###Almacenar DB en Drive

In [ ]:
# Copy db saved in colab into your drive

# drive.mount('/content/drive')
# !cp findata/finagg.sqlite /content/drive/MyDrive/
# import shutil

# # Source path
# source = 'findata/finagg.sqlite'
# # Destination path
# destination = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'

# shutil.copy(source, destination)

###Inspect Tables

In [ ]:
from sqlalchemy import create_engine, inspect, MetaData

db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
# Combine all metadata
all_metadata = MetaData()
# Setup the database and create tables based on the combined metadata
engine = create_engine(f'sqlite:///{db_path}')
all_metadata.create_all(engine)

# Reflect the tables from the engine into the metadata
all_metadata.reflect(bind=engine)

# Get a list of all the tables
tables = all_metadata.tables.keys()

# Use an inspector to fetch columns and other details
inspector = inspect(engine)

#Ver datos
for table in tables:
    print("Table:", table)
    for column in inspector.get_columns(table):
        print("  Column:", column['name'], column['type'])

Table: fred.raw.series
  Column: series_id VARCHAR
  Column: realtime_start VARCHAR
  Column: realtime_end VARCHAR
  Column: date VARCHAR
  Column: value FLOAT
Table: fred.refined.economic
  Column: date VARCHAR
  Column: CIVPART FLOAT
  Column: LOG_CHANGE(CPIAUCNS) FLOAT
  Column: LOG_CHANGE(CSUSHPINSA) FLOAT
  Column: LOG_CHANGE(DJIA) FLOAT
  Column: FEDFUNDS FLOAT
  Column: LOG_CHANGE(GDP) FLOAT
  Column: LOG_CHANGE(GDPC1) FLOAT
  Column: GS10 FLOAT
  Column: LOG_CHANGE(M2) FLOAT
  Column: MICH FLOAT
  Column: LOG_CHANGE(NASDAQ100) FLOAT
  Column: LOG_CHANGE(NASDAQCOM) FLOAT
  Column: PSAVERT FLOAT
  Column: LOG_CHANGE(SP500) FLOAT
  Column: UMCSENT FLOAT
  Column: UNRATE FLOAT
  Column: LOG_CHANGE(WALCL) FLOAT
Table: fundam.refined.fundam
  Column: ticker VARCHAR
  Column: date VARCHAR
  Column: PriceBookRatio FLOAT
  Column: PriceEarningsRatio FLOAT
Table: sec.raw.submissions
  Column: cik VARCHAR
  Column: ticker VARCHAR
  Column: entity_type VARCHAR
  Column: sic VARCHAR
  Colum

###Metadata BEA
######Los datos de BEA no se cargan con el install. Los datos de nipa no se pudieron obtener

In [ ]:
from finagg.bea.sql import fixed_assets, gdp_by_industry, input_output, nipa
from sqlalchemy import create_engine

# Usar metadata para crear las tablas en tu base de datos
fixed_assets.metadata.create_all(engine)
gdp_by_industry.metadata.create_all(engine)
input_output.metadata.create_all(engine)
nipa.metadata.create_all(engine)


###Obtener Datos BEA

In [ ]:
df_gdp = finagg.bea.api.gdp_by_industry.get()
df_gdp.to_sql('gdp_by_industry', engine, if_exists='append', index=False)
df_gdp

In [ ]:
df_fix_ass = finagg.bea.api.fixed_assets.get()
df_fix_ass.to_sql('fixed_assets', engine, if_exists='append', index=False)
df_fix_ass

In [ ]:
df_in_out = finagg.bea.api.input_output.get()
df_in_out.to_sql('input_output', engine, if_exists='append', index=False)
df_in_out

,table_id,year,row_code,row_description,row_type,col_code,col_description,col_type,value
0,6000,1997,,Total commodity output,Commodity,311FT,Food and beverage and tobacco products,Commodity,2.673929e+00
1,6000,1997,,Total commodity output,Commodity,481,Air transportation,Commodity,1.951930e+00
2,6000,1997,,Total commodity output,Commodity,487OS,Other transportation and support activities,Commodity,1.653248e+00
3,6000,1997,,Total commodity output,Commodity,22,Utilities,Commodity,1.700775e+00
4,6000,1997,,Total commodity output,Commodity,493,Warehousing and storage,Commodity,1.354362e+00
...,...,...,...,...,...,...,...,...,...
587341,6010,2021,VAPRO,Value Added (producer prices),Commodity,22,Utilities,Industry,3.784360e+05
587342,6010,2021,VAPRO,Value Added (producer prices),Commodity,81,"Other services, except government",Industry,4.698510e+05
587343,6010,2021,VAPRO,Value Added (producer prices),Commodity,21,Mining,Industry,3.339350e+05
587344,6010,2021,VAPRO,Value Added (producer prices),Commodity,11,"Agriculture, forestry, fishing, and hunting",Industry,2.065680e+05


###Show Data

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    table_name = table[0]
    # Colocar el nombre de la tabla entre comillas dobles
    cursor.execute(f'SELECT COUNT(*) FROM "{table_name}";')
    count = cursor.fetchone()[0]
    print(f"Table {table_name} has {count} rows.")


Table fred.raw.series has 34527 rows.
Table fred.refined.economic has 2353 rows.
Table indices.raw.djia has 30 rows.
Table indices.raw.sp500 has 503 rows.
Table indices.raw.nasdaq100 has 101 rows.
Table sec.raw.submissions has 471 rows.
Table sec.raw.tags has 172005 rows.
Table sec.refined.annual has 1276 rows.
Table sec.refined.annual.normalized has 664 rows.
Table sec.refined.quarterly has 3519 rows.
Table sec.refined.quarterly.normalized has 3113 rows.
Table yfinance.raw.prices has 4257432 rows.
Table yfinance.refined.daily has 4039449 rows.
Table fundam.refined.fundam has 283344 rows.
Table fundam.refined.fundam.normalized has 263607 rows.


###Show Prices

In [ ]:
db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT * FROM 'yfinance.raw.prices' WHERE ticker='AAPL' LIMIT 50")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

('AAPL', '1980-12-12', 0.0994495078921318, 0.09988187306116328, 0.0994495078921318, 0.0994495078921318, 469033600.0)
('AAPL', '1980-12-15', 0.09469354958662465, 0.09469354958662465, 0.09426118433475494, 0.09426118433475494, 175884800.0)
('AAPL', '1980-12-16', 0.08777497940939089, 0.08777497940939089, 0.08734261989593506, 0.08734261989593506, 105728000.0)
('AAPL', '1980-12-17', 0.08950439840555191, 0.08993676354420131, 0.08950439840555191, 0.08950439840555191, 86441600.0)
('AAPL', '1980-12-18', 0.09209936112165451, 0.09253172053349129, 0.09209936112165451, 0.09209936112165451, 73449600.0)
('AAPL', '1980-12-19', 0.09772007167339325, 0.09815243689955126, 0.09772007167339325, 0.09772007167339325, 48630400.0)
('AAPL', '1980-12-22', 0.10247607529163359, 0.10290921414264524, 0.10247607529163359, 0.1024760752916336, 37363200.0)
('AAPL', '1980-12-23', 0.1068004295229912, 0.10723279463678612, 0.1068004295229912, 0.10680042952299118, 46950400.0)
('AAPL', '1980-12-24', 0.1124211847782135, 0.112853

###Distinct tags from SEC

In [ ]:
db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT DISTINCT tag FROM 'sec.raw.tags'")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

('Assets',)
('AssetsCurrent',)
('EarningsPerShareBasic',)
('InventoryNet',)
('LiabilitiesCurrent',)
('NetIncomeLoss',)
('StockholdersEquity',)
('CommonStockSharesOutstanding',)
('Liabilities',)
